## Part 1
---
Part 1 asks us to start at zero and get to a target binary number through the minimum number of XOR operations with predefined states.

For now, the joltages don't play a part so they will not be loaded when reading.

In [79]:
with open('puzzle_input.txt', 'r') as f:
    rows = [row.split(' ')[:-1] for row in f.read().splitlines()]
    rows = [[c[1:-1].split(',') for c in row] for row in rows]

## Solution
---

The best way to think about this is through the use of an example: 

`[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}` (Notice that since in part 1 we don't need the voltages, we can safely ignore them) 

This essentially asks, starting from zero '....' what's the minimum amount of buttons we need to press to get to the target '.##.'?
To solve this, we will use the intuition from the question that discussed how the work as booleans through the XOR operation `^` as pressing the same button twice causes us to go back to where we started.

The target can be rephrased as a binary number where '#' is 1 and '.' is zero, this then becomes '0110'. Since the target is 4 digits long, there's a maximum of 2^4 = 16 possible states we can get to.

The buttons can also be represented as binary strings, with (3) mapping to '0010' as this is the bit that it edits.

The buttons in bits are then: `['0110', '0101', '0010', '0011', '1010', '1100']`

We can now create a graph with 2^4 = 16 nodes and loop over all nodes to create edges between them based on where the buttons take you from a given node.

See picture below for a graph depicting this exact thing for the example problem.

![](sample_graph.png)


The solution for the example is then the shortest path from zero '0000' = 0 to the target node '0110' = 6.

Repeating this for all problems in the input and summing the shortest paths gives the solution.

In [80]:
from bitarray import bitarray
from bitarray.util import ba2int
import networkx as nx

# Turns the target into a bitvector
def bin_target(target: str) -> str:
    target = target[0]

    binary = ""
    for i in range(len(target)):
        if target[i] == '#':
            binary += '1'

        else:
            binary += '0'

    binary = bitarray(binary)
    return binary

# Turns the buttons into a list of bitvectors
def bin_buttons(buttons: list[str], length: int) -> list[str]:
    binary_list = []
    for button in buttons:
        button_binary = ""
        for i in range(length):
            if str(i) in button:
                button_binary += '1'

            else:
                button_binary += '0'
        binary_list.append(bitarray(button_binary))
        
    return binary_list

problems = []
for row in rows:
    target = row[0]
    buttons = row[1:]
    problems.append([bin_target(target), bin_buttons(buttons, len(target[0]))])

shortest_path = []
for problem in problems:
    target = problem[0]
    
    buttons = problem[1]
    
    G = nx.Graph()
    
    for state in range(2**len(target)):
        for button in buttons:
            G.add_edge(state, state ^ ba2int(button))
    
    path = nx.shortest_path(G, source=0, target = ba2int(target))
    shortest_path.append(len(path) - 1)

print(sum(shortest_path))

545


## Part 2
---
Part two is an integer linear programming problem that asks us to meet a certain amount of jolt requirements which must be met based on what buttons are pressed and what joltages those buttons affect. In short:
1. Each button affects a subset of the jolt requirements and increases the jolts by one every time it's pressed
2. Each jolt requirement specifies the amount of jolts necessary at that index
3. We will minimise the amount of buttons pressed

First we must change the way we load the data.


In [55]:
with open('puzzle_input.txt', 'r') as f:
    rows = [row.split(' ')[1:] for row in f.read().splitlines()]
    rows = [[c[1:-1].split(',') for c in row] for row in rows]
    rows = [[[int(c) for c in lst] for lst in row] for row in rows]


To solve the integer linear programming problem we will use the z3 solver.

In [56]:
from z3 import *

#Based on 3.3 Wood workshop, linear programming and Leonid Kantorovich from SAT/SMT by Example by Dennis Yurichev

total = 0
for row in rows:
    #a will contain the buttons as a list of lists
    a = row[:-1]

    #b will contain the jolt requirements as a list
    b = row[-1]

    #Since we are trying to minimise the number of presses
    s = Optimize()

    #Variable we will try to minimise
    button_presses = Int('button_presses')

    #Dynamically generating buttons and jolts as they can vary in size
    buttons = [Int(f"button_{c}") for c in range(len(a))]
    jolts = [Int(f"jolt_{c}") for c in range(len(b))]

    #We know the number of presses is the sum of the amount of times a button was pressed
    s.add(button_presses == sum(buttons))

    #The number of times pressed must be positive
    s.add([b >= 0 for b in buttons])

    #The jolts must match those defined in the jolt requirements from b
    s.add([jolts[i] == b[i] for i in range(len(jolts))])
    
    '''
    Dynamically generating constraints depending on which buttons affect which voltages i.e. 
    if jolt_0 depends only on buttons {1,3} then this will say jolt_0 = sum(# of times buttons{1,3} were pressed)
    '''
    for j in range(len(b)):
        lst = []
        for i in range(len(a)):
            if j in a[i]:
                lst.append(buttons[i])
        s.add(jolts[j] == sum(lst))
    
    s.minimize(button_presses)
    
    if s.check() == sat:
        m = s.model()
    
        presses = m[button_presses].as_long()
        total += presses

print(total)

22430
